<a href="https://colab.research.google.com/github/gnloop/MFA-Universal-Notebook/blob/slicer/utau_conversion/Arpasing_Conversion-TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Montreal Forced Aligner](https://montreal-forced-aligner.readthedocs.io/)**
_Command line utility for forced alignment using Kaldi_

\
____
\

## _Tool to automatically convert Arpasing voicebanks into datasets for DiffSinger_

# Known issues:
### - No support for 'q' phoneme
\
____


# Huge thanks to PixPrucer and HAI-D for basically making every part of this notebook. I just updated things around basically ⛹

In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Condacolab
#@markdown The session will crash and restart, that's normal!
!pip install -q condacolab
import condacolab
condacolab.install()
from IPython.display import clear_output
clear_output()
print("All done, please wait for the session to restart!")

In [ ]:
#@title Install MFA
!conda install -c conda-forge montreal-forced-aligner spacy sudachipy sudachidict-core
!pip install speechbrain
!mfa model download acoustic english_us_arpa
from IPython.display import clear_output
clear_output()
print("All done!")

In [ ]:
#@title Unzip VB and generate transcriptions
#@markdown Unzip your voicebank, make sure it is a zip file with ONLY .wav files and an index.csv file! (one pitch only)

file_location = '/content/drive/MyDrive/wav.zip' #@param {type:"string"}

!7z x "$file_location" -o/content/db

import csv
import os
import re

# Path to the index.csv file and the /content/db folder
index_file = '/content/db/index.csv'
db_folder = '/content/db'

# Open the index.csv file for reading
with open(index_file, 'r') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Iterate over each row in the CSV file
    for row in reader:
        # Extract the wav filename and the corresponding text
        wav_filename = row[0]
        text = row[1]

        # Create the full path for the output txt file
        txt_filepath = os.path.join(db_folder, wav_filename[:-4] + '.txt')  #[:-4] part is to remove ".wav"

        # Open the txt file for writing
        with open(txt_filepath, 'w') as txtfile:
            # Write the text to the txt file
            txtfile.write(text)


# Path to the output folder for custom.dict
output_folder = '/root/Documents/MFA/pretrained_models/dictionary'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Define the vowel sequences to modify (including IY)
vowel_sequences = ["AA", "AE", "AH", "AO", "AX", "EH", "IH", "UH", "UW", "AW", "AY", "EY", "OW", "OY", "IY", "ER"]

# Create and write to the custom.dict file
with open(os.path.join(output_folder, 'custom.dict'), 'w') as f:
    for filename in os.listdir(db_folder):
        if filename.endswith('.txt'):
            # Read the content of the text file
            with open(os.path.join(db_folder, filename), 'r') as txtfile:
                content = txtfile.read().strip()

            # Create the modified content
            modified_content = content.replace('_', ' ').upper()

            # Add "1" after specific vowel sequences in the modified content
            for sequence in vowel_sequences:
                modified_content = modified_content.replace(sequence, sequence + "1")

            # Write the original and modified content to the custom.dict file
            f.write(f"{content}\t{modified_content}\n")

from IPython.display import clear_output
clear_output()
print("Wavs extracted, transcriptions and custom dictionary generated. To check if it worked, go to /root/Documents/MFA/pretrained_models/dictionary")

In [ ]:
#@title Start aligning!
!mfa align /content/db custom english_us_arpa /content/alignment --beam 400
from IPython.display import clear_output
clear_output()
print("All done! Check the 'alignment' folder for .TextGrid files")

In [ ]:
#@title Install TextGrid to LAB converter
!pip install mytextgrid
!git clone https://github.com/gnloop/MFA-Universal-Notebook
!mv /content/MFA-Universal-Notebook/text2lab_test.py /content/alignment/
from IPython.display import clear_output
clear_output()
print("All done!")

In [ ]:
#@title Convert TextGrid to LAB
!python -X utf8 /content/alignment/text2lab_test.py -c /content/MFA-Universal-Notebook/converters/converter_EN-ARPA.txt
from IPython.display import clear_output
clear_output()
print("Your .lab files should now be under the 'alignment' folder!")


In [ ]:
#@title Zip output
# @markdown Enter the path where you want your labels saved (no / at the end)
zip_path = '/content/drive/MyDrive/MFA_output' #@param {type:"string"}
from IPython.display import display, HTML, Javascript
import os

# Create the directory if it doesn't exist
os.makedirs(zip_path, exist_ok=True)

# Zip labels
!zip -j {zip_path}/labels.zip /content/alignment/*.lab

from IPython.display import clear_output
clear_output()
print("You can now download your labels in the labels.zip file and wavs in the sliced_wavs.zip file!")